In [6]:
tb.RequiredConditionV1?

Init signature:
tb.RequiredConditionV1(
    data: Optional[toloka.client.project.template_builder.base.BaseComponentOrAny] = None,
    *,
    hint: Optional[toloka.client.project.template_builder.base.BaseComponentOrAny] = None,
    version: Optional[str] = '1.0.0',
) -> None
Docstring:     
Checks that the data is filled in. This way you can get the Toloker to answer all the required questions.

If used inside the validation property, you can omit the data property and the same property will be used from the
parent component field (the one that contains the condition.required component).

Attributes:
    data: Data to be filled in. If the property is not specified, the data of the parent component (the one that
        contains condition.required) is used.
    hint: Validation error message that a Toloker will see.

Examples:
    How to check that image is uploaded.

    >>> image = tb.fields.MediaFileFieldV1(
    >>>     tb.data.OutputData('image'),
    >>>     tb.fields.MediaFileFie

In [2]:
import yaml

In [3]:
import os
import datetime
import logging
import sys
import time
import getpass
import pandas 
import requests

from textwrap import dedent

import toloka.client as toloka
import toloka.client.project.template_builder as tb

# False - if problems with tasks in toloka-kit still occure
USE_KIT_FOR_TASKS = False 

PASSWORD = os.environ.get(
    "TOLOKA_PWD", 
    getpass.getpass('Enter your OAuth token: ')
)

# PRODUCTION or SANDBOX, SANDBOX is default
ENVIRONMENT = os.environ.get("TOLOKA_ENV", "SANDBOX")

N_TASKS = os.environ.get("N_TASKS", 2)

logging.basicConfig(
    format='[%(levelname)s] %(name)s: %(message)s',
    level=logging.DEBUG,
    stream=sys.stdout,
)


/home/fatuus/deepschool-cvr-toloka/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: Interrupted by user

In [5]:
# tb.MediaFileFieldV1?
tb.FileFieldV1?

Init signature:
tb.FileFieldV1(
    data: Optional[toloka.client.project.template_builder.base.BaseComponent] = None,
    accept: Optional[toloka.client.project.template_builder.base.BaseComponentOrListBaseComponentOrStr] = None,
    *,
    multiple: Optional[toloka.client.project.template_builder.base.BaseComponentOrbool] = None,
    hint: Optional[toloka.client.project.template_builder.base.BaseComponentOrAny] = None,
    label: Optional[toloka.client.project.template_builder.base.BaseComponentOrAny] = None,
    validation: Optional[toloka.client.project.template_builder.base.BaseComponent] = None,
    version: Optional[str] = '1.0.0',
) -> None
Docstring:     
This component can be used for uploading files. It's displayed in the interface as an upload button.

You can restrict the file types to upload in the "accept" property. By default, only one file can be uploaded,
but you can allow multiple files in the "multiple" property.

If a Toloker logs in from a mobile device, it's more 

In [ ]:
CFG_PATH = os.environ.get("", 2)

In [2]:
toloka_client = toloka.TolokaClient(
    PASSWORD, ENVIRONMENT
)
print(toloka_client.get_requester())


[DEBUG] httpx: load_ssl_context verify=True cert=None trust_env=True http2=False
[DEBUG] httpx: load_verify_locations cafile='/home/fatuus/deepschool-cvr-toloka/venv/lib/python3.10/site-packages/certifi/cacert.pem'
[DEBUG] httpcore.connection: connect_tcp.started host='sandbox.toloka.dev' port=443 local_address=None timeout=10.0 socket_options=None
[DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7f7030d4c520>
[DEBUG] httpcore.connection: start_tls.started ssl_context=<ssl.SSLContext object at 0x7f706826d4c0> server_hostname='sandbox.toloka.dev' timeout=10.0
[DEBUG] httpcore.connection: start_tls.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7f706831e140>
[DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'GET']>
[DEBUG] httpcore.http11: send_request_headers.complete
[DEBUG] httpcore.http11: send_request_body.started request=<Request [b'GET']>
[DEBUG] httpcore.http11: send_request_

### Preparing Web UI

In [3]:
# tb.FileFieldV1(
#     data=tb.OutputData("image"),
#     label="barcode photo",
#     validation=tb.RequiredConditionV1(),
# )

In [4]:


# image_loader = tb.MediaFileFieldV1(
#     tb.OutputData('image'),
#     tb.MediaFileFieldV1.Accept(photo=True),
#     label="Photo of a barcode with it's number",
#     validation=tb.RequiredConditionV1(),
#     multiple=False,
# )

image_loader = tb.FileFieldV1(
    data=tb.OutputData("image"),
    label="barcode photo",
    validation=tb.RequiredConditionV1(),
)

# How performers will see the task
project_interface = toloka.project.TemplateBuilderViewSpec(
    view=tb.ListViewV1([image_loader]),
    settings={
        'showSubmit': True,
        'showFinish': True,
        'showTimer': True,
        'showReward': True,
        'showTitle': True,
        'showRoute': True,
        'showComplain': True,
        'showMessage': True,
        'showSubmitExit': True,
        'showFullscreen': True,
        'showInstructions': True,
    },
)


### Creating Project and Data Specifications

In [5]:
input_specification = {"id": toloka.project.IntegerSpec(required=True, hidden=False)}
output_specification = {"image": toloka.project.FileSpec()}
new_project = toloka.Project(
    public_name="Barcode Photo Search (toloka-kit)",
    public_description="Take a photo of a barcode with its number",
    task_spec=toloka.project.task_spec.TaskSpec(
        input_spec=input_specification,
        output_spec=output_specification,
        view_spec=project_interface,
    ),
    public_instructions = dedent(
        """
        <style scoped> 	.ugc ul { 	list-style-type: disc; 	} 	ul, ol { 	padding-left: 15px; 	margin-left: 25px; 	} 	li { 	margin: 2px 0; 	padding-right: 10px; 	} </style><p>Help us develop a barcode recognition system. Your task is to take a photo of a barcode with it's number.</p> <h3>Requirements for photos</h3> <ul><li><b>Number is required.</b> Barcodes that do not have numbers a not acceptable.</li><li><b>Digit only number.</b> The number of a barcode must contain only digits. The exception is symbol "&gt;" in the end.</li><li><b>Readability.</b>&nbsp;Barcode (lines) and it's number must be recognizable.</li><li><b>Uniqueness.</b> Please do not upload photos from internet. Non unique photos will not be accepted.<br></li><li><b>Quality.</b>&nbsp;Minimal photo resolution is 960&nbsp;× 720. The photo must not contain any watermarks, text, or logos that are not part of the product.</li><li><b>Realism.</b> Computer graphics, images, and freeze-frames from videos are not considered photos.</li><li><b>Single barcode.</b> Photo must contain only one barcode.</li><li><b>Distance.</b> Barcode with it's number must occupy from 50% to 90% of photo.</li></ul> <h3>Review of completed tasks</h3> <p>We usually check tasks within a few days. The maximum review period is 21 days.</p><p>The photos that don’t meet the requirements, will be rejected.<br></p> <h3>Bad examples</h3><p>Here are bad examples of barcodes:</p><p></p><ul><li>Barcode without a number&nbsp;<img src="https://disk.yandex.ru/i/KxMm2odHF3cSyQ" alt=""><a rel="noopener noreferrer" href="https://disk.yandex.ru/i/KxMm2odHF3cSyQ" style="background-color:rgb( 255 , 255 , 255 )" target="_blank">https://disk.yandex.ru/i/KxMm2odHF3cSyQ</a></li><li>Barcode with invalid number (must contain only digits):&nbsp;<a rel="noopener noreferrer" href="https://disk.yandex.ru/i/9EdV_8TWS_EjuA" target="_blank">https://disk.yandex.ru/i/9EdV_8TWS_EjuA</a></li><li>Barcode with invalid number (must contain only digits):&nbsp;&nbsp;<a rel="noopener noreferrer" href="https://disk.yandex.ru/i/ofnje7RX4Hs16g" style="background-color:rgb( 255 , 255 , 255 )" target="_blank">https://disk.yandex.ru/i/ofnje7RX4Hs16g</a></li></ul><p></p><p><br></p><h3>Good examples</h3><p>Here are conditionally good example, the only problem - they are not realistic:</p><p></p><ul><li>Symbol "&gt;" in the end is OK: <a rel="noopener noreferrer" href="https://disk.yandex.ru/i/4_IzlPiFTlcNng" target="_blank">https://disk.yandex.ru/i/4_IzlPiFTlcNng</a></li><li><a rel="noopener noreferrer" href="https://disk.yandex.ru/i/O5DlAGYHZUBzOg" target="_blank">https://disk.yandex.ru/i/O5DlAGYHZUBzOg</a></li></ul><p></p><p><br></p>
        """
    )
)


In [6]:
# new_project.public_instructions = """
# [English version is below]
# Возьмите 

# Take a picture of your pet if it is a cat or a dog and select the appropriate label type.<br><br>
# If you don't have a cat or a dog, take a photo of anything and select a "Not a cat nor a dog" label. There should be exactly one animal in the photo, clearly visible, not cropped. The animal can be photographed from any side and in any position. You can take a picture of a pet in your arms.<br><br>
# It should be clearly visible what animal is depicted (e.g. do not photograph your pet's back in the dark).
# """

In [7]:
new_project = toloka_client.create_project(new_project)


[DEBUG] httpcore.connection: close.started
[DEBUG] httpcore.connection: close.complete
[DEBUG] httpcore.connection: connect_tcp.started host='sandbox.toloka.dev' port=443 local_address=None timeout=10.0 socket_options=None
[DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7f7030eef190>
[DEBUG] httpcore.connection: start_tls.started ssl_context=<ssl.SSLContext object at 0x7f706826d4c0> server_hostname='sandbox.toloka.dev' timeout=10.0
[DEBUG] httpcore.connection: start_tls.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7f7030eef0a0>
[DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: send_request_headers.complete
[DEBUG] httpcore.http11: send_request_body.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: send_request_body.complete
[DEBUG] httpcore.http11: receive_response_headers.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: r

### Creating a pool

In [8]:
# Create a pool
new_pool = toloka.Pool(
    project_id=new_project.id,
    private_name='Pool 1',
    may_contain_adult_content=False,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),
    reward_per_assignment=0.01, # USD
    auto_accept_solutions=False,
    auto_accept_period_day=10,
    assignment_max_duration_seconds=60*10,
    defaults=toloka.Pool.Defaults(default_overlap_for_new_task_suites=1)
)

new_pool = toloka_client.create_pool(new_pool)


[DEBUG] httpcore.connection: close.started
[DEBUG] httpcore.connection: close.complete
[DEBUG] httpcore.connection: connect_tcp.started host='sandbox.toloka.dev' port=443 local_address=None timeout=10.0 socket_options=None
[DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7f7030eef0a0>
[DEBUG] httpcore.connection: start_tls.started ssl_context=<ssl.SSLContext object at 0x7f706826d4c0> server_hostname='sandbox.toloka.dev' timeout=10.0
[DEBUG] httpcore.connection: start_tls.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7f7030eeec20>
[DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: send_request_headers.complete
[DEBUG] httpcore.http11: send_request_body.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: send_request_body.complete
[DEBUG] httpcore.http11: receive_response_headers.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: r

In [11]:
if USE_KIT_FOR_TASKS:
    tasks = [
        toloka.Task(input_values={"id": id}, pool_id=new_pool.id)
        for id in range(N_TASKS)
    ]
    toloka_client.create_tasks(tasks, allow_defaults=False)
    print(f'Populated pool with {len(tasks)} tasks')
    print(f'To view this pool, go to https://sandbox.toloka.yandex.com/requester/project/{new_project.id}/pool/{new_pool.id}')
else:
    if ENVIRONMENT == "PRODUCTION":
        url = "https://toloka.dev/api/v1/tasks?skip_invalid_items=true"
    else:
        url = "https://sandbox.toloka.dev/api/v1/tasks?skip_invalid_items=true"
    headers = {
      'Authorization': f"OAuth {PASSWORD}",
      'Content-Type': 'application/json'
    }
    payload = {
      "input_values": {
        "id": 1
      },
      "pool_id": new_pool.id,
      "overlap": 1,
    }
    response = requests.post(url, headers=headers, json=payload)
    print(response.text)


[DEBUG] urllib3.connectionpool: Starting new HTTPS connection (1): sandbox.toloka.dev:443
[DEBUG] urllib3.connectionpool: https://sandbox.toloka.dev:443 "POST /api/v1/tasks?skip_invalid_items=true HTTP/1.1" 201 None
{"id":"0000170fb8--647b93d1ff14706a9f93a9bb","input_values":{"id":1},"pool_id":"1511352","overlap":1,"infinite_overlap":false,"reserved_for":[],"unavailable_for":[],"created":"2023-06-03T19:26:09.985","remaining_overlap":1}


In [ ]:
if ENVIRONMENT == "PRODUCTION":
    url = "https://toloka.dev/api/v1/tasks"
else:
    url = "https://sandbox.toloka.dev/api/v1/tasks"
headers = {
  'Authorization': f"OAuth {PASSWORD}",
  'Content-Type': 'application/json'
}
payload = [
    {
        "input_values": {
            "id": "1"
        },
        "pool_id": new_pool.id,
        "overlap": 1,
    }
]
response = requests.post(url, headers=headers, json=payload)
print(response.text)


In [12]:
new_pool.id

'1511352'